<a href="https://colab.research.google.com/github/mo3tawesome/TweetsClassifier/blob/master/kFoldsValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving offenseval-training-v1.tsv to offenseval-training-v1 (2).tsv


In [0]:
import numpy as np
from keras.models import Sequential
from keras import layers
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
with open('offenseval-training-v1.tsv', 'r', encoding="utf-8") as f:
    reader = csv.reader(f, dialect='excel', delimiter='\t')
    results = [] #tsv file 0:id, 1:sentence, 2: off/not
    sentences = []
    lables = []
    for row in reader:
        results.append(row)
        sentences.append(row[1])
        lables.append(row[2])
print(results[1][1])
count=0
removal_list = ["@USER"]
import re
for sentence in sentences:
  edit_string_as_list = sentence.split()
  final_list = [word for word in edit_string_as_list if word not in removal_list]
  final_string = ' '.join(final_list)
  #final_string = re.sub("^[a-z][a-z\s]*$", "", final_string) #remove non word chars
  sentences[count]=final_string.lower()
  count+=1
  
  
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Main settings
epochs = 2
embedding_dim = 100
maxlen = 100
output_file = 'data/output.txt'



# Train-test split
sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, lables, test_size=0.25, random_state=1000)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# Tokenize words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1
# Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

    # Parameter grid for grid search
param_grid = dict(num_filters=[190,200,250],
                      kernel_size=[3,4,5],
                      vocab_size=[vocab_size],
                      embedding_dim=[50],
                      maxlen=[maxlen])
model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, batch_size=10,
                            verbose=False)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=1, n_iter=5)
grid_result = grid.fit(X_train, y_train)

    # Evaluate testing set
test_accuracy = grid.score(X_test, y_test)






@USER She should ask a few native Americans what their take on this is.
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 35.7min finished


In [0]:

print(grid_result.best_score_)
print(grid_result.best_params_)
print(test_accuracy)

0.758509566290501
{'vocab_size': 17963, 'num_filters': 250, 'maxlen': 100, 'kernel_size': 5, 'embedding_dim': 50}
0.761703411408041
